In [1]:
!pip install sumy parsel pycountry tensorflow_hub nltk transformers tensorflow

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.3 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=86dc9a400b9d649991fef602c7ea45c396a9acab9cd8f314813239b8b990cd24
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6b0e1f40834cc422b0477c9c133bb1394cddd246e2a95f0201c55be9ea3dd8b6
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built breadability docopt


In [2]:
import gc
import pandas as pd
from transformers import pipeline
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from tqdm import tqdm
from parsel import Selector
import time
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
import pycountry
import string
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models, callbacks

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# 1. Data Collection

In [3]:
!git clone -n --depth=1 --filter=tree:0 \
  https://github.com/CoronaNetDataScience/corona_tscs

!cd corona_tscs && git sparse-checkout set --no-cone /data/CoronaNet/data_country/coronanet_release && git checkout

Cloning into 'corona_tscs'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 1 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1/1), done.
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 0), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 19.90 KiB | 3.98 MiB/s, done.
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 201 (delta 37), reused 1 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (201/201), 27.02 MiB | 2.37 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (201/201), done.
Your branch is up to date with 'origin/master'.


In [4]:
import pandas as pd
import os
dfs = []
for csv in os.listdir('corona_tscs/data/CoronaNet/data_country/coronanet_release'):
    # read each csv into a separate dataframe
    df = pd.read_csv(os.path.join('corona_tscs/data/CoronaNet/data_country/coronanet_release', csv), low_memory=False)
    dfs.append(df)

big_frame = pd.concat(dfs, ignore_index=True)

In [16]:
big_frame['type'].unique()

array(['New Task Force, Bureau or Administrative Configuration',
       'Closure and Regulation of Schools', 'COVID-19 Vaccines', 'Curfew',
       'Declaration of Emergency', 'External Border Restrictions',
       'Health Monitoring', 'Health Resources', 'Health Testing',
       'Hygiene', 'Internal Border Restrictions', 'Lockdown',
       'Other Policy Not Listed Above', 'Public Awareness Measures',
       'Quarantine', 'Restriction and Regulation of Businesses',
       'Restriction and Regulation of Government Services',
       'Restrictions of Mass Gatherings', 'Social Distancing',
       'Anti-Disinformation Measures'], dtype=object)

In [22]:
big_frame['type'].unique() #after map

array(['Government Measures', 'Education', 'Health', 'Restrictions',
       'Other', 'Awareness'], dtype=object)

In [21]:
type_mapping = {
    # Health
    "Hygiene": "Health",
    "Health Monitoring": "Health",
    "Health Testing": "Health",
    "COVID-19 Vaccines": "Health",
    "Health Resources": "Health",

    # Education
    "Closure and Regulation of Schools": "Education",


    # Restrictions
    "Quarantine": "Restrictions",
    "Social Distancing": "Restrictions",
    "Lockdown": "Restrictions",
    "Curfew": "Restrictions",
    "Internal Border Restrictions": "Restrictions",
    'External Border Restrictions': "Restrictions",
    'Restriction and Regulation of Businesses': "Restrictions",
    "Restriction and Regulation of Government Services": "Restrictions",
    "Restrictions of Mass Gatherings": "Restrictions",

    # Government Measures
    "Declaration of Emergency": "Government Measures",
    "New Task Force, Bureau or Administrative Configuration": "Government Measures",

    # Awareness
    "Public Awareness Measures": "Awareness",
    "Anti-Disinformation Measures": "Awareness",

    # Other
    "Other Policy Not Listed Above": "Other",

}

# Apply mapping
big_frame['type'] = big_frame['type'].map(type_mapping)

In [23]:
big_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200486 entries, 0 to 200485
Data columns (total 68 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   record_id               200486 non-null  object 
 1   policy_id               200486 non-null  object 
 2   entry_type              200486 non-null  object 
 3   update_type             49350 non-null   object 
 4   update_level            31382 non-null   object 
 5   update_level_var        28034 non-null   object 
 6   description             200486 non-null  object 
 7   date_announced          200481 non-null  object 
 8   date_start              200486 non-null  object 
 9   date_end                129455 non-null  object 
 10  date_end_spec           199383 non-null  object 
 11  country                 200486 non-null  object 
 12  ISO_A3                  199851 non-null  object 
 13  ISO_A2                  199680 non-null  object 
 14  init_country_level  

In [24]:
gc.collect()

del(dfs)

In [25]:
cols = ['description', 'date_end_spec', 'country', 'init_country_level', 'enforcer']
big_frame[cols] = big_frame[cols].astype(str)
big_frame['descriptions'] = big_frame[cols].agg(' '.join, axis=1)


In [26]:
big_frame['descriptions'] = big_frame['descriptions'].astype(str)
df = big_frame[["descriptions", "type"]].drop_duplicates(subset=['descriptions']).reset_index(drop=True)

In [27]:
df.rename(columns={"descriptions": "description"}, inplace=True)

In [28]:
df

,description,type
0,"September 14, 2020: With Bulletin N. 262, the ...",Government Measures
1,"On April 7, New Zealand government announced t...",Government Measures
2,"In Bulgaria, as of March 11, all schools and e...",Education
3,"As of March 11, 2020, the Security Council in ...",Education
4,"As of March 11, 2020, children, students, and ...",Education
...,...,...
119268,Non contact sports outside to continue in Rwan...,Restrictions
119269,"On June 02,Rwandan government announces that M...",Restrictions
119270,All resumed businesses in Rwanda MUST adhere t...,Restrictions
119271,Rwandans urged to Continue observing the instr...,Restrictions


In [29]:
import random
import nltk
from nltk.corpus import wordnet
# Download NLTK resources (only first time)
nltk.download('wordnet')
nltk.download('omw-1.4')
import random
import nltk
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import pandas as pd
import numpy as np
import random
import nltk
from nltk.corpus import wordnet
from sklearn.utils import shuffle

nltk.download("wordnet")
nltk.download("omw-1.4")
import pandas as pd



# -----------------------------------------------------
# STEP 1: Shuffle the dataset
# -----------------------------------------------------
df = shuffle(df, random_state=42).reset_index(drop=True)

# -----------------------------------------------------
# Helper functions for text augmentation
# -----------------------------------------------------

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():
            if lem.name().lower() != word.lower():
                synonyms.add(lem.name().replace("_", " "))
    return list(synonyms)

def synonym_replacement(sentence, n=1):
    words = sentence.split()
    if len(words) < 2:
        return sentence

    candidates = [w for w in words if get_synonyms(w)]
    if not candidates:
        return sentence

    for _ in range(n):
        word = random.choice(candidates)
        synonym = random.choice(get_synonyms(word))
        words = [synonym if w == word else w for w in words]

    return " ".join(words)

def random_deletion(sentence, p=0.1):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    new_words = [w for w in words if random.random() > p]
    return " ".join(new_words) if new_words else random.choice(words)

def random_swap(sentence, n=1):
    words = sentence.split()
    if len(words) < 2:
        return sentence

    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]

    return " ".join(words)

def augment_text(text):
    choice = random.choice(["synonym", "swap", "delete"])

    if choice == "synonym":
        return synonym_replacement(text, n=1)
    elif choice == "swap":
        return random_swap(text, n=1)
    elif choice == "delete":
        return random_deletion(text, p=0.15)

    return text

# -----------------------------------------------------
# STEP 2: Oversample minority classes with augmentation
# -----------------------------------------------------

class_counts = df["type"].value_counts()
max_count = class_counts.max()

augmented_rows = []

for label, count in class_counts.items():
    subset = df[df["type"] == label]
    needed = max_count - count

    if needed > 0:
        for i in range(needed):
            row = subset.sample(1).iloc[0]
            new_desc = augment_text(row["description"])
            augmented_rows.append({"description": new_desc, "type": label})

aug_df = pd.DataFrame(augmented_rows)

# -----------------------------------------------------
# STEP 3: Combine and shuffle again
# -----------------------------------------------------
balanced_df = shuffle(pd.concat([df, aug_df], ignore_index=True), random_state=42)

balanced_df.head()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,description,type
337591,Burundi's Health Minister urges the Public to ...,Awareness
421442,"In Papua New Guinea ""Mandatory mask wearing in...",Other
288847,To reduce the spread of COVID-19 within South ...,Government Measures
408960,"Grenada (04.17.2020): ""Prime Minister and Mini...",Other
108812,"On February 2, 2020, the Fujian Provincial Dep...",Restrictions


In [32]:
balanced_df

,description,type
337591,Burundi's Health Minister urges the Public to ...,Awareness
421442,"In Papua New Guinea ""Mandatory mask wearing in...",Other
288847,To reduce the spread of COVID-19 within South ...,Government Measures
408960,"Grenada (04.17.2020): ""Prime Minister and Mini...",Other
108812,"On February 2, 2020, the Fujian Provincial Dep...",Restrictions
...,...,...
259178,"In Texas on May 5, 2020 ""Chief Kidd and Commis...",Government Measures
365838,"Starting from April 8, 2020, everyone governme...",Awareness
131932,"On April 4, the governor of the Stavropol' Kra...",Health
146867,"On January 25th, 2020 the Federal Office of Pu...",Health


## 2. Preprocess Data

In [33]:
# Combine all preprocessing steps

def preprocess_text(text):

    # 1. Convert text to lowercase
    def to_lower(text):
        return text.lower()

    # 2. Remove punctuation and numbers
    def remove_punctuation_numbers(text):
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'\b(\d+)(st|nd|rd|th)\b', '', text)  # Remove Ordered Numbering
        return text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation

    # 3. Tokenize text into words
    def tokenize(text):
        return nltk.word_tokenize(text)

    # 4. Remove stopwords (common words with little meaning)
    stop_words = set(stopwords.words('english'))
    def remove_stopwords(tokens):
        return [word for word in tokens if word not in stop_words]

    # 5. Apply lemmatization to reduce words to their base (dictionary) form
    lemmatizer = WordNetLemmatizer()
    def apply_lemmatization(tokens):
        return [lemmatizer.lemmatize(word) for word in tokens]

    text = to_lower(text)
    text = remove_punctuation_numbers(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = apply_lemmatization(tokens)
    return " ".join(tokens)

# 6- Apply Preprocessing
balanced_df["description"] = balanced_df["description"].apply(preprocess_text)

In [38]:
stop_words = list(stopwords.words('english'))

# include country names in stopwords
country_text = []
for text in balanced_df["description"].tolist():
    for c in pycountry.countries:
        if c.name.lower() in text:
            text = re.sub(c.name.lower(), '', text)
    country_text.append(text)

balanced_df["description"] = country_text

for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    balanced_df['description'] = balanced_df['description'].str.replace(regex_stopword, '')

In [39]:
balanced_df

,description,type
337591,health minister urge public maintain hygiene ...,Awareness
421442,papua new mandatory mask wearing port effecti...,Other
288847,reduce spread covid within south march state ...,Government Measures
408960,prime minister minister finance dr right hono...,Other
108812,february fujian provincial department civil af...,Restrictions
...,...,...
259178,texas may chief kidd commissioner task provide...,Government Measures
365838,starting april everyone government launched wh...,Awareness
131932,april governor stavropol kray ordered plan ope...,Health
146867,january th federal office public health infor...,Health


In [ ]:
df2

,description,type
311161,athens th athens municipal daily mass disinfe...,Hygiene
151357,nb april department education early released ...,Closure and Regulation of Schools
172181,ia new arrival expected selfisolate one locati...,Quarantine
49906,meeting february czech government announced ne...,Curfew
289526,march announced n set virtual map developed an...,Health Monitoring
...,...,...
119879,august government republic adopted decision l...,External Border Restrictions
259178,april diagnostic capacity expanded northern r...,Health Testing
131932,may ministry health chuvashia announced doctor...,Health Resources
146867,april reminded grave sweeper maintain appropr...,Social Distancing


# Modelling

#### Label encoding

In [41]:
category_codes = {'Health' : 0, 'Education': 1, 'Restrictions' : 2, 'Awareness' : 3, 'Government Measures' : 4, 'Other' : 5

}

# Category mapping
balanced_df['type'] = balanced_df['type'].map(category_codes)
balanced_df.head()

,description,type
337591,health minister urge public maintain hygiene ...,3
421442,papua new mandatory mask wearing port effecti...,5
288847,reduce spread covid within south march state ...,4
408960,prime minister minister finance dr right hono...,5
108812,february fujian provincial department civil af...,2


#### Split train-test sets

In [ ]:
df2

,description,type,type_combined,type_code
311161,ministry health informs following enter group ...,Hygiene,Health,Hygiene
151357,starting march n government announced nonforma...,Closure and Regulation of Schools,Education,Closure and Regulation of Schools
172181,march national government day quarantine stat...,Quarantine,Restrictions,Quarantine
49906,february health departement canton ticino iss...,Social Distancing,Restrictions,Social Distancing
289526,announced july government launched website ex...,Health Monitoring,Health,Health Monitoring
...,...,...,...,...
119879,october guangxi bucolic government require peo...,External Border Restrictions,Restrictions,External Border Restrictions
259178,noting november greek government imposed price...,Health Testing,Health,Health Testing
131932,april city dresden begin distribute face mask ...,Health Resources,Health,Health Resources
146867,march government saarland announced policy so...,Social Distancing,Restrictions,Social Distancing


In [50]:
balanced_df

,description,type
337591,health minister urge public maintain hygiene ...,3
421442,papua new mandatory mask wearing port effecti...,5
288847,reduce spread covid within south march state ...,4
408960,prime minister minister finance dr right hono...,5
108812,february fujian provincial department civil af...,2
...,...,...
259178,texas may chief kidd commissioner task provide...,4
365838,starting april everyone government launched wh...,3
131932,april governor stavropol kray ordered plan ope...,0
146867,january th federal office public health infor...,0


In [51]:
df2=balanced_df.copy()

In [52]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# 1. Prepare One Hot Encoder
ohe = OneHotEncoder(sparse_output=False)  # sparse=False for numpy array output

# 2. Fit-transform the column
y_ohe = ohe.fit_transform(df2[['type']])

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df2['description'],      # Inputs
    y_ohe,                   # One-hot encoded labels
    test_size=0.2,
    random_state=42,
    stratify=df2['type']   # Same rule as before
)


In [53]:
df2

,description,type
337591,health minister urge public maintain hygiene ...,3
421442,papua new mandatory mask wearing port effecti...,5
288847,reduce spread covid within south march state ...,4
408960,prime minister minister finance dr right hono...,5
108812,february fujian provincial department civil af...,2
...,...,...
259178,texas may chief kidd commissioner task provide...,4
365838,starting april everyone government launched wh...,3
131932,april governor stavropol kray ordered plan ope...,0
146867,january th federal office public health infor...,0


In [54]:
#TF Hub model
embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")

X_train_embed = embed(X_train.tolist())
X_test_embed = embed(X_test.tolist())

In [55]:
df2['type'].value_counts()

,count
type,
3,75539
5,75539
4,75539
2,75539
1,75539
0,75539


In [60]:
# Build a tuned model
import tensorflow as tf
from tensorflow.keras import layers, callbacks

import tensorflow as tf
from tensorflow.keras import layers, regularizers, callbacks

model = tf.keras.Sequential([
    # Input
    layers.Input(shape=(128,), dtype=tf.float32),
    # Block 1
    layers.Dense(512, activation='relu',
                 kernel_initializer='he_normal',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.4),

    # Block 2
    layers.Dense(256, activation='relu',
                 kernel_initializer='he_normal',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.3),

    # Block 3

    layers.Dense(128, activation='gelu',
                 kernel_initializer='he_normal',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.2),

    layers.Dense(6, activation='softmax')
])


optimizer = tf.keras.optimizers.AdamW(learning_rate=3e-4, weight_decay=1e-4)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])



early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

checkpoint = callbacks.ModelCheckpoint(
    "best_model.keras",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    X_train_embed, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/50
5096/5099 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6459 - loss: 1.1353
Epoch 1: val_loss improved from inf to 0.80964, saving model to best_model.keras
5099/5099 ━━━━━━━━━━━━━━━━━━━━ 59s 11ms/step - accuracy: 0.6460 - loss: 1.1352 - val_accuracy: 0.7600 - val_loss: 0.8096 - learning_rate: 3.0000e-04
Epoch 2/50
5094/5099 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7514 - loss: 0.8313
Epoch 2: val_loss improved from 0.80964 to 0.70675, saving model to best_model.keras
5099/5099 ━━━━━━━━━━━━━━━━━━━━ 58s 11ms/step - accuracy: 0.7514 - loss: 0.8313 - val_accuracy: 0.7922 - val_loss: 0.7068 - learning_rate: 3.0000e-04
Epoch 3/50
5095/5099 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7781 - loss: 0.7396
Epoch 3: val_loss improved from 0.70675 to 0.63892, saving model to best_model.keras
5099/5099 ━━━━━━━━━━━━━━━━━━━━ 56s 11ms/step - accuracy: 0.7781 - loss: 0.7396 - val_accuracy: 0.8140 - val_loss: 0.6389 - learning_rate: 3.0000e-04
Epoch 4/50
5098/5099 ━━━━━━━━━━━━━━

# Evaluation

In [63]:
y_pred = model.predict(X_test_embed)
y_pred_labels = tf.argmax(y_pred, axis=1)

print(classification_report(tf.argmax(y_test, axis=1), y_pred_labels))

2833/2833 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step
              precision    recall  f1-score   support

           0       0.90      0.88      0.89     15107
           1       0.97      0.98      0.98     15108
           2       0.93      0.86      0.90     15108
           3       0.93      0.98      0.95     15108
           4       0.96      0.98      0.97     15108
           5       0.95      0.98      0.97     15108

    accuracy                           0.94     90647
   macro avg       0.94      0.94      0.94     90647
weighted avg       0.94      0.94      0.94     90647

